In [36]:
class OptionPricing:
    def __init__(self, stock_name, strike_price, ttm, option_type = 'Call'): #visualizaition = True):
        self.stock_name = stock_name
        self.option_type = option_type
        #self.visual = visualization
        self.strike_price = strike_price
        self.ttm = ttm
        
    
    def get_stock_info(self, start = 0, end = 0):

        import pandas as pd
        import numpy as np
        import datetime
        import pandas_datareader.data as web
        import fix_yahoo_finance as yf
        end = datetime.date.today()
        start = end - datetime.timedelta(days = 365*10)
        try:
            data = web.get_data_yahoo(self.stock_name, start,end)
            data['Return'] = data['Close'].pct_change()
            data = data.dropna()
            self.std = data['Return'].describe().loc['std']
            self.std = self.std * np.sqrt(252) #annualize
            self.price = data['Close'].iloc[len(data)-1]

            return self.price, self.std

        except:
            print('Cannot find stock data')
    
    
    
    def get_rf(self, date = 0, tenor = '10 yr'):
        import pandas as pd
        import requests
        from bs4 import BeautifulSoup
        import datetime
        url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
        response = requests.get(url)

        if response.status_code == 200:
            data_page = BeautifulSoup(response.content,'lxml')
            yield_table = data_page.find('table', class_ = 't-chart')
            trs = yield_table.find_all('tr')
            col = [trs[0].find_all('th', scope = 'col')[i].get_text() 
                    for i in range(1,len(trs[0].find_all('th', scope = 'col')))]
            index = [trs[i].find_all('td', scope = 'row')[0].get_text() for i in range(1,len(trs))]

            rates = [[float(trs[i].find_all('td')[j].get_text()) for j in range(1,len(col)+1)] for i in range(1,len(trs))]
            data = pd.DataFrame(rates, columns = col, index = index)
            data = data/100 #pct_value

            date = data.index.values[len(data)-1] #latest data

            self.rf_rate = data.loc[date][tenor]
        else:
            print('Failure')

        return self.rf_rate #risk free rate
    
    
    def BS_model(self):

        stock_price = self.price # from get_stock_info
        var = self.std ** 2
        strike_price = self.strike_price
        r_f = self.r_f
        ttm =self.ttm
        
        return self.BS_df 

        
    
    def BS_sensitivity_plot(self):
        stock_price = self.price
        var = self.std ** 2
        strike_price = self.strike_price
        r_f = self.r_f
        ttm =self.ttm
        option_type = self.option_type
        
        
        
        
#     def mc_euro_options(self, option_type,S,K,T,r,v,simulations):
        

    
#     def simulations_converge(self, option_type,S,K,T,r,v,simulations):
#         pass
    
#     def stock_time(self, S,K,r,v,simulations,steps,time_delta):
#         pass
    
#     def BinomialTreeCRR(self, T, S, K, r, sigma, n, option_type='C'):
#         pass

In [41]:
O = OptionPricing('AAPL', 100, 1 )
O.get_stock_info()
O.get_rf()

0.0301